In [1]:
from tqdm import tqdm
from models import CompletionNetwork, ContextDiscriminator
from datasets import ImageDataset
from losses import completion_network_loss
from utils import (
    gen_input_mask,
    gen_hole_area,
    crop,
    sample_random_batch,
    poisson_blend,
)
from torch.utils.data import DataLoader
from torch.optim import Adadelta, Adam
from torch.nn import BCELoss, DataParallel
from torchvision.utils import save_image
from PIL import Image
import torchvision.transforms as transforms
import torch
import random
import os
import argparse
import numpy as np
import json
import time

In [6]:
data_path = "/home/guest2/gandata1"

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('data_dir')
parser.add_argument('result_dir')
parser.add_argument('--recursive_search', action='store_true', default=False)
parser.add_argument('--init_model_cn', type=str, default=None)
parser.add_argument('--init_model_cd', type=str, default=None)
parser.add_argument('--steps_1', type=int, default=100)
parser.add_argument('--steps_2', type=int, default=100)
parser.add_argument('--steps_3', type=int, default=100)
parser.add_argument('--snaperiod_1', type=int, default=10000)
parser.add_argument('--snaperiod_2', type=int, default=2000)
parser.add_argument('--snaperiod_3', type=int, default=10000)
parser.add_argument('--max_holes', type=int, default=1)
parser.add_argument('--hole_min_w', type=int, default=48)
parser.add_argument('--hole_max_w', type=int, default=96)
parser.add_argument('--hole_min_h', type=int, default=48)
parser.add_argument('--hole_max_h', type=int, default=96)
parser.add_argument('--cn_input_size', type=int, default=160)
parser.add_argument('--ld_input_size', type=int, default=96)
parser.add_argument('--optimizer', type=str, choices=['adadelta', 'adam'], default='adadelta')
parser.add_argument('--bsize', type=int, default=16)
parser.add_argument('--bdivs', type=int, default=1)
parser.add_argument('--data_parallel', action='store_true')
parser.add_argument('--num_test_completions', type=int, default=16)
parser.add_argument('--mpv', nargs=3, type=float, default=None)
parser.add_argument('--alpha', type=float, default=4e-4)
parser.add_argument('--arc', type=str, choices=['celeba', 'places2'], default='celeba')

_StoreAction(option_strings=['--arc'], dest='arc', nargs=None, const=None, default='celeba', type=<class 'str'>, choices=['celeba', 'places2'], help=None, metavar=None)

In [5]:
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--recursive_search]
                             [--init_model_cn INIT_MODEL_CN]
                             [--init_model_cd INIT_MODEL_CD]
                             [--steps_1 STEPS_1] [--steps_2 STEPS_2]
                             [--steps_3 STEPS_3] [--snaperiod_1 SNAPERIOD_1]
                             [--snaperiod_2 SNAPERIOD_2]
                             [--snaperiod_3 SNAPERIOD_3]
                             [--max_holes MAX_HOLES] [--hole_min_w HOLE_MIN_W]
                             [--hole_max_w HOLE_MAX_W]
                             [--hole_min_h HOLE_MIN_H]
                             [--hole_max_h HOLE_MAX_H]
                             [--cn_input_size CN_INPUT_SIZE]
                             [--ld_input_size LD_INPUT_SIZE]
                             [--optimizer {adadelta,adam}] [--bsize BSIZE]
                             [--bdivs BDIVS] [--data_parallel]
                             [--num_test_completions NUM_TE

SystemExit: 2

/home/guest2/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
# ================================================
# Preparation
# ================================================
args.data_dir = os.path.expanduser(args.data_dir)
args.result_dir = os.path.expanduser(args.result_dir)
if args.init_model_cn != None:
    args.init_model_cn = os.path.expanduser(args.init_model_cn)
if args.init_model_cd != None:
    args.init_model_cd = os.path.expanduser(args.init_model_cd)
if torch.cuda.is_available() == False:
    raise Exception('At least one gpu must be available.')
else:
    gpu = torch.device('cuda:0')

# create result directory (if necessary)
if os.path.exists(args.result_dir) == False:
    os.makedirs(args.result_dir)
for s in ['phase_1', 'phase_2', 'phase_3']:
    if os.path.exists(os.path.join(args.result_dir, s)) == False:
        os.makedirs(os.path.join(args.result_dir, s))


NameError: name 'args' is not defined